In [ ]:
import itertools

class Map:
    def __init__(self, n_rows, n_cols):
        self.map = [["." for _ in range(n_cols)] for _ in range(n_rows)]
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.guard_start = None
        self.obstructions = []

    def __repr__(self):
        # if self.map is None:
        #     return "Map is not initialized."
        return "\n".join("".join(row) for row in self.map)
    
    @classmethod
    def load_from_file(cls, filename):
        with open(filename, "r") as f:
            map_loading = []
            for line in f:
                line = line.strip()
                map_loading.append(list(line))
        n_rows = len(map_loading)
        n_cols = len(map_loading[0])
        obj = cls(n_rows, n_cols)
        
        for row, row_list in enumerate(map_loading):
            for col, char in enumerate(row_list):
                if char == "#":
                    obj.place_obstruction(row, col)
                elif char in "<>^v":
                    direction_dict = {"^":0, ">":1, "v":2, "<":3}
                    direction = direction_dict[char]
                    obj.guard_start = (row, col, direction)
                    obj.place_guard(row, col, direction)
        
        return obj

    def is_obstruction(self, row, col):
        if self.map[row][col] == "#":
            return True
        else:
            return False
    
    def place_obstruction(self, row: int, col: int) -> None:
        self.map[row][col] = "#"
        self.obstructions.append((row, col))
    
    def place_guard(self, row, col, direction):
        DIRECTION_SYMBOL = {0: "^", 1: ">", 2: "v", 3: "<"}
        self.map[row][col] = DIRECTION_SYMBOL[direction]

    def num_loop_intro_spots(self) -> int:
        count = 0
        loop_combos = []
        obstruction_triplets = self._get_obstruction_triplets()
        for triplet in obstruction_triplets:
            triplet = list(triplet)
            if self._check_row_criterion(triplet) and self._check_col_criterion(triplet):
                count += 1
                loop_combos.append(triplet)
        return count, loop_combos

    def _get_obstruction_triplets(self) -> list[tuple]:
        return list(itertools.combinations(self.obstructions, 3))
    
    def _check_row_criterion(self, obstruction_tripelts: list[tuple]) -> bool:
        obstruction_tripelts.sort()
        row_distances = []
        for i in range(1, len(obstruction_tripelts)):
            row_distances.append(obstruction_tripelts[i][0]-obstruction_tripelts[i-1][0])
        #print(f"Row Distances: {row_distances}")
        if 1 in row_distances:
            return True #l.index(1) # this number is either 0 or 1
        else:
            return False

    def _check_col_criterion(self, obstruction_triplets: list[tuple]) -> bool:
        obstruction_triplets.sort(key=lambda x: x[1])
        col_distances = []
        for i in range(1, len(obstruction_triplets)):
            col_distances.append(obstruction_triplets[i][1]-obstruction_triplets[i-1][1])
        #print(f"Col Distances: {col_distances}")
        if 1 in col_distances:
            return True
        else:
            return False

    


class Guard:
    # Directions are North, East, South, West
    DIRECTIONS = [(-1,0), (0,1), (1,0), (0,-1)]
    DIRECTION_SYMBOL = {0: "^", 1: ">", 2: "v", 3: "<"}

    def __init__(self, row, col, direction):
        self.row: int = row
        self.col: int = col
        self.direction: int = direction
        self.path: list[tuple] = []
        self.symbol: str = self.DIRECTION_SYMBOL[direction]
        self.on_map: bool = True

    def move(self, game_map: Map) -> None:
        d_row, d_col = self.DIRECTIONS[self.direction]
        row_new, col_new = self.row + d_row, self.col + d_col
            
        if row_new < 0 or row_new == game_map.n_rows or col_new < 0 or col_new == game_map.n_cols:
            self.path.append((self.row, self.col))
            self.on_map = False
            print(f"Guard left the map at position ({self.row},{self.col})")
        
        elif game_map.is_obstruction(row_new, col_new):
            print(f"On position ({self.row},{self.col}): Hit an obstruction, turned right")
            self.rotate()

        else:
            self.path.append((self.row, self.col))
            game_map.map[self.row][self.col] = "X"
            self.row = row_new
            self.col = col_new
            game_map.place_guard(self.row, self.col, self.direction)

    def rotate(self) -> None:
        self.direction = (self.direction + 1) % 4
        self.symbol = self.DIRECTION_SYMBOL[self.direction]
        print(f"Guard turned right. New direction is: {self.direction} ({self.symbol})")

    def unique_positions_in_path(self) -> int:
        uniques = []
        for elem in self.path:
            if elem not in uniques:
                uniques.append(elem)
        return len(uniques)
    
    @property
    def moves_in_loop(self) -> bool:
        pass
            

class Game:
    pass

### Solving Part 1

In [ ]:
my_map = Map.load_from_file("day6_input.txt")
print(my_map)
print(my_map.guard_start)
r, c, d = my_map.guard_start # r, c, d = row, col, direction
my_guard = Guard(r, c, d)
while my_guard.on_map:
#for i in range(10):
    my_guard.move(my_map)
print(my_map)
print(my_guard.unique_positions_in_path())


### Towards solving part 2
- generate all permutations of 3 obstructions
- write a function taht checks if the three suffice for catching the guard in a loop
    - order the obstructions by the first elemnt of each tuple
    - first elemnts have to be n, n+1, m, m+1 with m > n+1
    - second elemnts have to be a < b, c = b-1, d < c, d = a-1

In [35]:
my_test_map = Map.load_from_file("day6_example_input.txt")
print(my_test_map.obstructions)
print(my_test_map)
c, l = my_test_map.num_loop_intro_spots()
print(c)
print(l)

[(0, 4), (1, 9), (3, 2), (4, 7), (6, 1), (7, 8), (8, 0), (9, 6)]
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
12
[[(0, 4), (7, 8), (1, 9)], [(6, 1), (7, 8), (1, 9)], [(8, 0), (7, 8), (1, 9)], [(6, 1), (3, 2), (4, 7)], [(3, 2), (4, 7), (7, 8)], [(3, 2), (9, 6), (4, 7)], [(6, 1), (3, 2), (7, 8)], [(6, 1), (4, 7), (7, 8)], [(8, 0), (4, 7), (7, 8)], [(8, 0), (9, 6), (4, 7)], [(8, 0), (6, 1), (7, 8)], [(8, 0), (6, 1), (9, 6)]]


In [34]:
for i in range(len(l)):
    print(f"Element {i} of found hits")
    new_map = Map(10,10)
    for elem in l[i]:
        new_map.place_obstruction(elem[0], elem[1])
    print(new_map)
    print("\n\n")

Element 0 of found hits
....#.....
.........#
..........
..........
..........
..........
..........
........#.
..........
..........



Element 1 of found hits
..........
.........#
..........
..........
..........
..........
.#........
........#.
..........
..........



Element 2 of found hits
..........
.........#
..........
..........
..........
..........
..........
........#.
#.........
..........



Element 3 of found hits
..........
..........
..........
..#.......
.......#..
..........
.#........
..........
..........
..........



Element 4 of found hits
..........
..........
..........
..#.......
.......#..
..........
..........
........#.
..........
..........



Element 5 of found hits
..........
..........
..........
..#.......
.......#..
..........
..........
..........
..........
......#...



Element 6 of found hits
..........
..........
..........
..#.......
..........
..........
.#........
........#.
..........
..........



Element 7 of found hits
..........
......

In [5]:
import itertools

combinations = list(itertools.combinations(my_test_map.obstructions, 3))
print(len(combinations))
print(combinations)

test1 = list(combinations[0])
#test1.sort()
print(test1)

def check_row_criterion(l: list) -> bool:
    l.sort()
    row_distances = []
    for i in range(1, len(l)):
        row_distances.append(l[i][0]-l[i-1][0])
    print(f"Row Distances: {row_distances}")
    if 1 in row_distances:
        return True #l.index(1) # this number is either 0 or 1
    else:
        return False

def check_col_criterion(l: list) -> bool:
    l.sort(key=lambda x: x[1])
    col_distances = []
    for i in range(1, len(l)):
        col_distances.append(l[i][1]-l[i-1][1])
    print(f"Col Distances: {col_distances}")
    if 1 in col_distances:
        return True
    else:
        return False

check_row_criterion(test1)

56
[((0, 4), (1, 9), (3, 2)), ((0, 4), (1, 9), (4, 7)), ((0, 4), (1, 9), (6, 1)), ((0, 4), (1, 9), (7, 8)), ((0, 4), (1, 9), (8, 0)), ((0, 4), (1, 9), (9, 6)), ((0, 4), (3, 2), (4, 7)), ((0, 4), (3, 2), (6, 1)), ((0, 4), (3, 2), (7, 8)), ((0, 4), (3, 2), (8, 0)), ((0, 4), (3, 2), (9, 6)), ((0, 4), (4, 7), (6, 1)), ((0, 4), (4, 7), (7, 8)), ((0, 4), (4, 7), (8, 0)), ((0, 4), (4, 7), (9, 6)), ((0, 4), (6, 1), (7, 8)), ((0, 4), (6, 1), (8, 0)), ((0, 4), (6, 1), (9, 6)), ((0, 4), (7, 8), (8, 0)), ((0, 4), (7, 8), (9, 6)), ((0, 4), (8, 0), (9, 6)), ((1, 9), (3, 2), (4, 7)), ((1, 9), (3, 2), (6, 1)), ((1, 9), (3, 2), (7, 8)), ((1, 9), (3, 2), (8, 0)), ((1, 9), (3, 2), (9, 6)), ((1, 9), (4, 7), (6, 1)), ((1, 9), (4, 7), (7, 8)), ((1, 9), (4, 7), (8, 0)), ((1, 9), (4, 7), (9, 6)), ((1, 9), (6, 1), (7, 8)), ((1, 9), (6, 1), (8, 0)), ((1, 9), (6, 1), (9, 6)), ((1, 9), (7, 8), (8, 0)), ((1, 9), (7, 8), (9, 6)), ((1, 9), (8, 0), (9, 6)), ((3, 2), (4, 7), (6, 1)), ((3, 2), (4, 7), (7, 8)), ((3, 2),

True

In [ ]:
is_loopable = [(0,4), (1,9), (7,8)] # (6,3), pos 4
is_loopable2 = [(3,2), (4,7), (6,1)] # (7,6), pos 3
is_loopable3 = [(6,1), (9,6), (8,0)] # (7,7) pos 2
is_not_loopable = [(0, 4), (1, 9), (3, 2)]

tests = [is_loopable, is_loopable2, is_loopable3, is_not_loopable]

for test in tests:
    print(f"\nTesting: {test}")
    if check_col_criterion(test) and check_row_criterion(test):
        print("we can catch the guard in a loop")
    else:
        print("we cannot catch the guard in a loop")